<a href="https://colab.research.google.com/github/bhyanx/Uso-de-api-GEMINI/blob/main/Muestreo_API_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTACIÓN DE LIBRERIAS**


*   PANDAS
*   MATPLOTLIB
*   SEABORN
*   GOOGLE-GENERATIVEAI
*   GRADIO

In [ ]:
!pip install -q pandas matplotlib seaborn google-generativeai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00


# **Codigo para manejo de datos estadisticos y consultas por medio de IA**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import google.generativeai as genai
import gradio as gr
import io
import os
import requests
import json
from IPython.display import display, HTML
import numpy as np

"""## Configuración de la API de Gemini AI"""
# Función para configurar la API de Gemini
def setup_gemini_api(api_key):
    genai.configure(api_key=api_key)
    # Verificar que la API esté configurada correctamente
    try:
        # Usar gemini-2.0-flash como modelo por defecto (la versión más reciente)
        model = genai.GenerativeModel('gemini-2.0-flash')
        response = model.generate_content("Responde en español: Hola, ¿puedes decirme brevemente qué puede hacer Gemini AI?")
        print("✅ API de Gemini configurada correctamente")
        return True
    except Exception as e:
        print(f"❌ Error al configurar la API de Gemini: {e}")
        return False


"""## Funciones para el análisis de datos"""
# Función para generar estadísticas descriptivas del CSV
def generate_csv_stats(df):
    buffer = io.StringIO()

    # Información general del dataframe
    buffer.write(f"### Información general del CSV\n")
    buffer.write(f"- Número de filas: {df.shape[0]}\n")
    buffer.write(f"- Número de columnas: {df.shape[1]}\n")
    buffer.write(f"- Columnas: {', '.join(df.columns.tolist())}\n\n")

    # Primeras filas para vista previa
    buffer.write("### Vista previa de los datos\n")
    buffer.write(df.head().to_string())
    buffer.write("\n\n")

    # Estadísticas descriptivas
    buffer.write("### Estadísticas descriptivas\n")
    # Filtrar solo columnas numéricas para las estadísticas
    numeric_df = df.select_dtypes(include=[np.number])
    if not numeric_df.empty:
        buffer.write(numeric_df.describe().to_string())
    else:
        buffer.write("No hay columnas numéricas para mostrar estadísticas.\n")

    # Valores faltantes
    buffer.write("\n\n### Valores faltantes\n")
    missing_values = df.isnull().sum()
    missing_df = pd.DataFrame({
        'Columna': missing_values.index,
        'Valores faltantes': missing_values.values,
        'Porcentaje': (missing_values.values / len(df)) * 100
    })
    buffer.write(missing_df.to_string(index=False))

    # Tipos de datos
    buffer.write("\n\n### Tipos de datos\n")
    dtypes_df = pd.DataFrame({
        'Columna': df.dtypes.index,
        'Tipo': df.dtypes.values
    })
    buffer.write(dtypes_df.to_string(index=False))

    return buffer.getvalue()


# Función para visualizar datos con gráficos básicos
def generate_visualizations(df):
    # Crear una carpeta para guardar los gráficos si no existe
    if not os.path.exists('visualizations'):
        os.makedirs('visualizations')

    plots_info = []

    # 1. Distribución de variables numéricas
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        for i, col in enumerate(numeric_cols[:5]):  # Limitar a 5 columnas para no crear demasiados gráficos
            plt.figure(figsize=(10, 6))
            sns.histplot(df[col].dropna(), kde=True)
            plt.title(f'Distribución de {col}')
            plt.tight_layout()
            filename = f'visualizations/dist_{col}.png'.replace(' ', '_')
            plt.savefig(filename)
            plt.close()
            plots_info.append({
                'title': f'Distribución de {col}',
                'filename': filename,
                'type': 'histogram'
            })

    # 2. Conteo de variables categóricas
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    if len(categorical_cols) > 0:
        for i, col in enumerate(categorical_cols[:3]):  # Limitar a 3 columnas
            plt.figure(figsize=(12, 8))
            value_counts = df[col].value_counts().head(10)  # Mostrar solo las 10 categorías más frecuentes
            sns.barplot(x=value_counts.index, y=value_counts.values)
            plt.title(f'Conteo de {col} (Top 10)')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            filename = f'visualizations/count_{col}.png'.replace(' ', '_')
            plt.savefig(filename)
            plt.close()
            plots_info.append({
                'title': f'Conteo de {col} (Top 10)',
                'filename': filename,
                'type': 'barplot'
            })

    # 3. Matriz de correlación (si hay suficientes variables numéricas)
    if len(numeric_cols) > 1:
        plt.figure(figsize=(12, 10))
        corr_matrix = df[numeric_cols].corr()
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', fmt='.2f')
        plt.title('Matriz de Correlación')
        plt.tight_layout()
        filename = 'visualizations/correlation_matrix.png'
        plt.savefig(filename)
        plt.close()
        plots_info.append({
            'title': 'Matriz de Correlación',
            'filename': filename,
            'type': 'heatmap'
        })

    return plots_info


# Función para consultar a Gemini AI sobre los datos
def query_gemini_about_data(query, df, api_key):
    try:
        genai.configure(api_key=api_key)
        # Usar gemini-2.0-flash para consultas rápidas
        model = genai.GenerativeModel('gemini-2.0-flash')

        # Preparar contexto para Gemini
        df_info = df.head(10).to_string()  # Primeras filas como muestra
        columns_info = "\n".join([f"- {col}: {df[col].dtype}" for col in df.columns])

        # Preparar estadísticas solo para columnas numéricas
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        stats = ""
        if len(numeric_cols) > 0:
            stats = "\n".join([f"- {col}: min={df[col].min()}, max={df[col].max()}, mean={df[col].mean()}"
                              for col in numeric_cols])

        # Para columnas categóricas, incluir los valores únicos (limitados)
        categorical_cols = df.select_dtypes(include=['object', 'category']).columns
        cat_info = ""
        if len(categorical_cols) > 0:
            cat_values = []
            for col in categorical_cols:
                unique_vals = df[col].unique()
                if len(unique_vals) <= 10:  # Si hay pocos valores únicos, mostrarlos todos
                    cat_values.append(f"- {col}: valores = {', '.join(str(x) for x in unique_vals)}")
                else:
                    # Mostrar solo los 5 más frecuentes
                    top_vals = df[col].value_counts().nlargest(5).index.tolist()
                    cat_values.append(f"- {col}: valores más frecuentes = {', '.join(str(x) for x in top_vals)}")
            cat_info = "\n".join(cat_values)

        # Crear un prompt estructurado para Gemini
        prompt = f"""
        Estoy analizando un conjunto de datos con la siguiente estructura:

        COLUMNAS:
        {columns_info}

        ESTADÍSTICAS DE COLUMNAS NUMÉRICAS:
        {stats}

        INFORMACIÓN DE COLUMNAS CATEGÓRICAS:
        {cat_info}

        MUESTRA DE DATOS:
        {df_info}

        A partir de estos datos, responde de manera detallada a la siguiente consulta:

        {query}

        Responde en español e incluye detalles y análisis específicos basados en los datos proporcionados.
        """

        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error al consultar a Gemini AI: {str(e)}\n\nVerifica que tu API key sea correcta y que estés usando el formato adecuado para la API de Gemini 2.0."


"""## Interfaz de usuario con Gradio"""
def create_gradio_interface():
    # Variables globales
    df = None

    def upload_csv(file):
        try:
            nonlocal df
            df = pd.read_csv(file.name)
            stats = generate_csv_stats(df)
            return df.head().to_html(), stats, "CSV cargado correctamente"
        except Exception as e:
            return None, f"Error al cargar el archivo: {str(e)}", "Error"

    def set_api_key(api_key):
        if setup_gemini_api(api_key):
            return "API de Gemini configurada correctamente", api_key
        else:
            return "Error al configurar la API de Gemini", ""

    def ask_gemini(query, api_key):
        nonlocal df
        if df is None:
            return "Primero debes cargar un archivo CSV"

        if not api_key:
            return "Primero debes configurar la API de Gemini"

        try:
            return query_gemini_about_data(query, df, api_key)
        except Exception as e:
            return f"Error al consultar a Gemini: {str(e)}"

    def generate_plots():
        nonlocal df
        if df is None:
            return "Primero debes cargar un archivo CSV", []

        try:
            plots_info = generate_visualizations(df)
            plot_files = [plot['filename'] for plot in plots_info]
            return "Visualizaciones generadas correctamente", plot_files
        except Exception as e:
            return f"Error al generar visualizaciones: {str(e)}", []

    with gr.Blocks(title="Analizador de CSV con Gemini AI") as app:
        gr.Markdown("# 📊 Analizador de CSV con Gemini AI")
        gr.Markdown("Carga un archivo CSV, configura tu API key de Gemini AI y realiza consultas sobre tus datos.")

        # Variable para almacenar la API key
        saved_api_key = gr.State("")

        with gr.Tab("Cargar Datos"):
            with gr.Row():
                with gr.Column(scale=1):
                    file_input = gr.File(label="Selecciona un archivo CSV")
                    upload_button = gr.Button("Cargar CSV")
                    upload_status = gr.Textbox(label="Estado", interactive=False)

                with gr.Column(scale=2):
                    data_preview = gr.HTML(label="Vista previa")
                    data_stats = gr.Textbox(label="Estadísticas", interactive=False, max_lines=20)

        with gr.Tab("Configurar API"):
            gemini_api_key = gr.Textbox(label="API Key de Gemini AI", type="password")
            api_button = gr.Button("Configurar API")
            api_status = gr.Textbox(label="Estado", interactive=False)

        with gr.Tab("Consultar Datos"):
            query_input = gr.Textbox(label="Consulta sobre los datos", placeholder="Ej: ¿Cuáles son las tendencias principales en estos datos?")
            query_button = gr.Button("Consultar a Gemini AI")
            query_result = gr.Markdown(label="Respuesta")

        with gr.Tab("Visualizaciones"):
            viz_button = gr.Button("Generar Visualizaciones")
            viz_status = gr.Textbox(label="Estado", interactive=False)
            viz_gallery = gr.Gallery(label="Gráficos", show_label=True, columns=2, rows=3, height=600)

        # Conectar eventos
        upload_button.click(upload_csv, inputs=[file_input], outputs=[data_preview, data_stats, upload_status])
        api_button.click(set_api_key, inputs=[gemini_api_key], outputs=[api_status, saved_api_key])
        query_button.click(ask_gemini, inputs=[query_input, saved_api_key], outputs=[query_result])
        viz_button.click(generate_plots, inputs=[], outputs=[viz_status, viz_gallery])

    return app


"""## Iniciar la aplicación"""
# Mostrar un mensaje de bienvenida y explicación
print("¡Bienvenido al Analizador de CSV con Gemini AI!")
print("Este notebook te permite:")
print("1. Cargar un archivo CSV")
print("2. Analizar automáticamente los datos")
print("3. Realizar consultas en lenguaje natural usando Gemini AI")
print("4. Generar visualizaciones para mejor comprensión de los datos")
print("\nPara comenzar, ejecuta las celdas anteriores y luego ejecuta la siguiente celda para iniciar la interfaz.")

# Iniciar la interfaz de Gradio
app = create_gradio_interface()
app.launch(share=True, debug=True)

"""
## Instrucciones de uso

1. **Cargar datos**:
   - Ve a la pestaña "Cargar Datos"
   - Selecciona un archivo CSV de tu computadora
   - Haz clic en "Cargar CSV"
   - Verás una vista previa y estadísticas básicas de tus datos

2. **Configurar API**:
   - Ve a la pestaña "Configurar API"
   - Ingresa tu API Key de Gemini AI
   - Haz clic en "Configurar API"
   - **Nota importante**: Asegúrate de usar una API key válida para Gemini 2.0

3. **Consultar datos**:
   - Ve a la pestaña "Consultar Datos"
   - Escribe una pregunta sobre tus datos en lenguaje natural
   - Haz clic en "Consultar a Gemini AI"
   - Obtén una respuesta basada en el análisis de tus datos

4. **Visualizar datos**:
   - Ve a la pestaña "Visualizaciones"
   - Haz clic en "Generar Visualizaciones"
   - Explora los gráficos generados automáticamente

## Ejemplos de consultas con el CSV de ventas:

- "¿Cuál es el producto más vendido en términos de unidades totales?"
- "¿Existe alguna correlación entre el precio y la satisfacción del cliente?"
- "¿Qué vendedor tiene el mejor desempeño en ventas?"
- "¿Cuáles son las tendencias de ventas por región?"
- "¿Las promociones tienen un impacto positivo en las ventas?"
- "¿Qué categoría de productos genera más ingresos?"
- "¿Hay algún patrón estacional en las ventas durante estos meses?"

## Solución de problemas:

- Si recibes un error al configurar la API, verifica:
  1. Que tu API key sea correcta
  2. Que tengas acceso a la API de Gemini 2.0
  3. Que hayas habilitado la API de Generative Language en tu proyecto de Google Cloud
"""

¡Bienvenido al Analizador de CSV con Gemini AI!
Este notebook te permite:
1. Cargar un archivo CSV
2. Analizar automáticamente los datos
3. Realizar consultas en lenguaje natural usando Gemini AI
4. Generar visualizaciones para mejor comprensión de los datos

Para comenzar, ejecuta las celdas anteriores y luego ejecuta la siguiente celda para iniciar la interfaz.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8a5665684680324b23.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
